In [3]:
import pandas as pd
import re

# CONFIGURACIÓN
file_path = '../data/raw/SAIC.xlsx' # Nombre corregido

# 1. CARGA
print(f"📂 Cargando archivo: {file_path}...")
# header=4 indica que los títulos están en la fila 5 (índice 4)
df = pd.read_excel(file_path, header=4)

# Renombramos columnas basado en su posición para evitar errores de escritura
# Asumimos el orden: Año Censal | Entidad | Actividad | Personal | Producción
new_columns = {
    df.columns[0]: 'Anio_Censal',
    df.columns[1]: 'Entidad',
    # La columna 2 (Actividad) la ignoramos
    df.columns[3]: 'Personal_Ocupado', # H001A
    df.columns[4]: 'Produccion_Bruta'  # A111A
}
df.rename(columns=new_columns, inplace=True)

# 2. LIMPIEZA
# Función para limpiar nombre de Entidad (ej "01 Aguascalientes" -> "Aguascalientes")
def clean_entidad(val):
    if pd.isna(val): return "Desconocido"
    val_str = str(val).strip()
    # Regex: Quita dígitos iniciales seguidos de espacio
    return re.sub(r'^\d+\s+', '', val_str)

# Función para asegurar que los valores sean numéricos
def clean_numeric(val):
    if pd.isna(val): return 0.0
    # Si es string, limpiamos comas o espacios
    if isinstance(val, str):
        val = val.replace(',', '').strip()
        if val == '' or val == '-': return 0.0
    return float(val)

df['Entidad'] = df['Entidad'].apply(clean_entidad)
df['Personal_Ocupado'] = df['Personal_Ocupado'].apply(clean_numeric)
df['Produccion_Bruta'] = df['Produccion_Bruta'].apply(clean_numeric)

# 3. CÁLCULO
# Fórmula: Personal / (Producción * 1000)
# La Producción ya viene en millones, al multiplicar por 1000 ajustamos la escala
df['Indicador_Productividad'] = df.apply(
    lambda row: row['Produccion_Bruta'] / row['Personal_Ocupado']*1000
    if row['Produccion_Bruta'] != 0 else 0.0, 
    axis=1
)

# 4. RESULTADO FINAL
cols_finales = ['Anio_Censal', 'Entidad', 'Personal_Ocupado', 'Produccion_Bruta', 'Indicador_Productividad']
df_final = df[cols_finales]

print("\n--- RESUMEN DEL CÁLCULO (Top 5) ---")
print(df_final.head())

print("\n--- EJEMPLO (Aguascalientes) ---")
print(df_final[df_final['Entidad'] == 'Aguascalientes'])

# Opcional: Guardar
# df_final.to_excel("Indicador_Censo_Economico_Calculado.xlsx", index=False)

📂 Cargando archivo: ../data/raw/SAIC.xlsx...

--- RESUMEN DEL CÁLCULO (Top 5) ---
  Anio_Censal               Entidad  Personal_Ocupado  Produccion_Bruta  \
0        2023        Aguascalientes          423999.0        536022.609   
1        2023       Baja California         1155112.0        978451.037   
2        2023   Baja California Sur          227389.0        152657.248   
3        2023              Campeche          193656.0        399970.518   
4        2023  Coahuila de Zaragoza          984646.0       1687584.570   

   Indicador_Productividad  
0              1264.207248  
1               847.061616  
2               671.348429  
3              2065.366000  
4              1713.899787  

--- EJEMPLO (Aguascalientes) ---
  Anio_Censal         Entidad  Personal_Ocupado  Produccion_Bruta  \
0        2023  Aguascalientes          423999.0        536022.609   

   Indicador_Productividad  
0              1264.207248  
